# Lab Week 9

## In the last lab, we merged our paired end reads that overlapped, mapped our reads to the reference genome, then converted to bam files for downstream processing. We did all this on small files to make the programs run faster and see each of the individual steps. Over the break, I ran the full fastq files through this process to map the millions of reads contained in each file, a process that took many hours and many computer cores for each file. Today we will continue to process these files to identify SNPs from the sequencing reads and count the number of reads supporting the reference and alternate alleles. 

1) To get started, Log into spydur from the terminal tab: `ssh spydur` then navigate to your named folder in the `~/shared` directory and make a new folder for `lab_9` (remember no spaces in file or folder names, this causes lots of problems in Unix!).



2) Next, navigate to `~/shared/bam` and find the names of the final bam file for your two sequencing samples. You will also need the names of the 2 files that end in .bai for your file. These are index files that help the computer quickly locate records in the gigantic bam file.  Write the code below to copy these files to your `lab_9` folder. Pro-tip: you can use the wildcard character `*` to copy any file name that matches a particular pattern. For example `cp ZP_10*` will copy all files that start with `ZP_10`(both the .bam and the .bai file)


In [ ]:
%% bash
#write code to copy the files to your lab_9 folder here (one line for each sample, use the * to your advantage!





3) Navigate back to your lab_9 folder and make sure the bam files are there using `ls`. It is really important that you are working in this folder for the rest of the lab! 

4) Next we need to make a file that has the full paths of both of our working bam files that are in our lab_9 folder. Below, the code uses asterisk wildcard like we introduced above. But now we only want the name of the .bam file, so we need to add more information. `ZP*` will match any filename that starts with `ZP`. Since we want only the .bam files, we need to use `ZP*.bam` to isolate only these file names that start with `ZP` and end in `.bam`. 

In [ ]:
%% bash 
ls $PWD/ZP*.bam > sample_files.txt

## Question: Look at the `sample_files.txt` that you just made above using `less`. What did that code that you just ran do?

### Answer here

## Question: If you had not included the .bam in the ZP*.bam part of the command, how many files would be listed?

### Answer here


5) Now we need to make a second file that has just the names of our two samples on separate lines using the UNIX tool `echo`, which basically means "print". We will tell it to print our two file names with a line break in between them. The special character `\n` says to start a new line. Look at the command below and edit it for your file names. Inside the quotes, replace with the two names of your samples but leave the `\n`, for example `"ZP_1\nZP_2"`


In [ ]:
%% bash
echo -e "first_sample\nsecond_sample" > sample_names.txt


6) Look at your sample_names.txt file with `less` to make sure it did what you wanted it to (each file names should be on a separate line with no extra characters. If this file is not correct, your following steps will not work!

7) Next we are going to start a job on the erickson partition of spydur so that you can use more memory and computer cores. Run the code below, which is the same as what we used last week to activate your session.

In [ ]:
%% bash

srun --pty -t 3:00:00 --mem=40G --partition erickson --ntasks-per-node=3 bash


8) For our first analysis we are going to find out how much sequencing data we actually successfully mapped to the genome in the bam files. The program we'll use is a function of samtools called `samtools depth`. The general format is `samtools depth input.bam > output.txt`

Edit the general code below to update with the file names of your input file name (the one file you worked with last week), and name the output file "samplename_depth.txt" (but update with the actual name of your sample!). You and your partner can each calculate depth for the file you were working with last week. This calculation might take a couple of minutes, but you can start reading ahead.

In [ ]:
%% bash
samtools depth input.bam > sample_name_depth.txt


## Question: take a look at your output file above using `less`. What does each line of the file contain? 

### Answer here

9) The third column of the file you just generated contains the number of sequencing reads mapped to each position in the genome.  We can calculate the average depth of coverage for each scaffold using something called a "for loop" in unix. A for loop takes a variable and repeats calculations over a range of values for that variable. In this case, the code is taking a variable called `i` and executing the code for `i=1`, `i=2`...`i=5`. The code below uses a program called awk to calculate an average of nubmers in column 3 (the `'{sum+=$3} END { print sum/NR}'` part of the code calculates this average. Copy this whole block of code and run it. 

In [ ]:
%% bash
for i in {1..5}; do
echo "calculating depth for scaffold " $i
   y=`grep Scaffold_${i} sample_name_depth.txt | awk '{sum+=$3} END { print sum/NR}'`
    echo "the average depth of scaffold" $i "is" $y
    done
    

## Question: why did the code loop through the numbers 1 to 5?
### Answer here:

## Question: What part of the code is extracting data for only one scaffold? 
### Answer here:

## Question: How does the average coverage of the different chromosomes compare? Which is the lowest? 

### Answer here

## Can you think of a reason one chromosome might have lower coverage than the others? Hint: think about what chromosome is not present equally in all individuals in most species, and how that would be reflected when pooling many individuals together.

### Answer here

10) Next we are going to generate what is called a "pileup file". This file takes all the mapping information from the bam files and summarizes it for every position in the genome. We are just going to create it for a small region of the genome (Scaffold_1:10000000-11000000) to speed up our future analysis. Complete the code below, assigning  a name for your pileup file and start running it, then answer the questions below while it runs. 

In [ ]:
%% bash
#fill in the full path to the reference genome 
REFERENCE=~/shared/zind_ref_genome.fasta

#make sure you are in your lab_9 folder!

#make mpileup file of your data--edit the name of the pileup file below
samtools mpileup -r Scaffold_1:10000000-11000000 -B -f $REFERENCE --bam-list sample_files.txt > sample1_sample2.pileup

While you are waiting for the code to run, go to this website: http://www.htslib.org/doc/samtools-mpileup.html and read about the pileup format to answer the following questions:

## Question:  Approximately how many lines will you expect to have in your pileup file given the region of the genome that we are looking at? 

### Answer here:

## Question: How are bases that match the reference base indicated in the pileup file? How are SNPs indicated?

### Answer here:


11) After your code has run, look at your mpileup file using `less -S` and answer the following question:

## Question: where do you see the first SNP and how do you know? 

### Answer here 

## Now count the number of lines in your pileup file with `wc -l`. Does it meet your expectation? Why might there be  a small difference?

### Answer here

12) Now we will actually call SNPs and indels in this small region of the genome. The code below uses a program called Varscan to actually identify variants. We set filters so that it requires variants to have a coverage of at least 10 reads and a minimum allele frequency of 0.05. The p-value filter refers to a statistical probability that the SNP is real. The SNPs and indels are called with two separate commands `mpileup2snp` and `mpileup2indel`. To run the code, you  just need to replace the name of the pileup file that you generated above on the first line of each block of code. Then copy and paste everything to your terminal.

In [ ]:
%% bash
#tell the computer how much memory to use
JAVAMEM=40g

#call SNPs
java -Xmx$JAVAMEM -jar ~/shared/sw/varscan/VarScan.v2.4.4.jar mpileup2snp sample1_sample2.pileup \
  --output-vcf 1 \
  --min-coverage 10 \
  --min-var-freq 0.05 \
  --p-value 0.1 \
  --vcf-sample-list sample_names.txt > raw_snps.vcf
  
#call indels

java -Xmx$JAVAMEM -jar ~/shared/sw/varscan/VarScan.v2.4.4.jar mpileup2indel sample1_sample2.pileup \
  --output-vcf 1 \
  --min-coverage 10 \
  --min-var-freq 0.05 \
  --p-value 0.1 \
  --vcf-sample-list sample_names.txt > raw_indels.vcf

## Question: Read the output that printed to the terminal when your code was done. How many SNPs and how many indels did it identify?
### Answer here: 

## Question: Now go to https://www.ebi.ac.uk/training/online/courses/human-genetic-variation-introduction/variant-identification-and-analysis/understanding-vcf-format/ to learn about the output file, vcf files. Complete the activity and take a screenshot when you are done.
### Paste your screenshot here. It will just show up as a link to a jpg file but will be inserted when you export to PDF at the end. 

13) The first several lines of the vcf file begin with # and have important information about the data that can be found in the file. Use `less -S raw_snp.vcf` to look at your raw SNP calls vcf file to answer the following questions. They will be very difficult if you don't add the -S to make things print in nice columns. If you are stuck on these questions, please talk to me because I want to be sure you understand this information!  

## Question: the vcf will store a lot of information about each variant in each sample. The key to decoding this information is provided in the header lines that start with ##FORMAT=. Look at the #FORMAT lines of your vcf. What are the names used in the format column to indicate the number of reference reads and alternate reads?

### Answer here

## Question: The true header line of the vcf file that names the columns starts with ##CHROM. From left to right, where are the columns located that actually contain the data on your two samples?

### Answer here

## Question: Look at the first SNP in your sample. Where in the genome is it located?

### Answer here

## Question: Now keep reading across to the column that starts with ` GT:GQ:SDP...`. This column tells you the order of information that you will read about each sample, separated by a colon ( `:`) between each piece of information. You identified the abbreviations for reference and alternate read counts in a question above. Which positions in this series of information will have your reference and alternate read counts?

### Answer here

## Question: Now keep reading across to your first sample. How many reads were found for the reference and alternate allele? These are the numbers in the positions you identified above.

### Answer here

## Question: the vcf file format automatically assigns a "genotype" to each sample. Why does it not make sense to assign genotypes in pooled sequencing?

### Answer here:

14) Now that we've gotten oriented to the information in a vcf file, we need to do further filtering to only use high-confidence SNPs. As mentioned previously, indels cause quite a few problems for SNP calling, so we want to get rid of the SNPs that are located near indels. Varscan has a built in filtering program that can remove these. Run the code below to see what happens.

In [ ]:
%% bash
java -Xmx$JAVAMEM -jar ~/shared/sw/varscan/VarScan.v2.4.4.jar filter raw_snps.vcf \
 --min-var-freq 0.05 \
 --indel-file  raw_indels.vcf \
 --output-file snps_no_indels.vcf

## Question: How many SNPs were removed because they were near indels and how many SNPs remain in your file? This information will be displayed when the code is done running.

### Answer here

15) We ran all of these commands on just a tiny region of the genome to make things easier for us. I ran the SNP calling on all of our samples over the weekend. Using all of the samples together increases the ability to find variants relative to using just two samples because there is so much more information to compare. But, the resulting vcf file has all 10 samples and is quite large, so it is currently stored in a compressed .gz format. To make these easier to work with, you are going to extract just the two samples you are working with from each of these variant files. We will use a program called `bcftools` to do this with a command called `view`. This program will use the list of sample names that you already made (`sample_names.txt`) to extract just those samples from the vcf file and save a smaller file. 

In [ ]:
%% bash
bcftools view -S sample_names.txt ~/shared/ZP_all_noindel.vcf.gz > subset_snps_no_indels.vcf

16) Now confirm that the previous command did what you wanted it to by checking the sample names present in the file using the command below. `query -l` is a shortcut taht extracts all the sample names. 

In [ ]:
%% bash
bcftools query -l subset_snps_no_indels.vcf

17) We also don't want to include SNPs in regions of the genome that contain repetitive sequences like retrotransposons, because mapping reads to these regions is imprecise and calling SNPs is not accurate. 

## Question: Why would it be hard to map reads and call SNPs in repetitive regions? (hint: think about the definition of repetitive and how the mapping algorithm works)

### Answer here: 

18) I have a file that contains the positions of all of the repetitive sequences in the Z. indianus genome. The file is in a `gff` format, which is basically just a table of chromsomes, start positions, and stop positions for regions of the genome. We can use a program called `bedtools` to remove all of these regions from our vcf file as well. This program essentially does a subtraction where it will remove any SNPs found within regions defined in the `~/shared/zind_ref_genome_repeats.gff` file. Run the code below


In [ ]:
%% bash
bedtools intersect -v -header \
    -a subset_snps_no_indels.vcf \
    -b ~/shared/zind_ref_genome_repeats.gff \
    > subset_snps_no_indels_no_repeats.vcf


## Question: Use `wc -l` to count the number of lines in your starting file and final file from the command above. How many lines are in each and approximately how many SNPs were removed due to being in repetitive regions?

### Answer here

19) Some SNPs are just going to be weird. They might have more than two alleles. They may not have very many reads supporting them or they may have way too many reads supporting them. We want to get rid of those SNPs too. The command below will filter any SNPs that have less than 15 or more than 100 reads supporting them in your dataset. This command will apply several filters and write a new vcf with `.recode` added at the end of the file name 

In [ ]:
%% bash
vcftools --vcf subset_snps_no_indels_no_repeats.vcf \
--min-meanDP 15 \
--max-meanDP 150 \
--min-alleles 2 \
--max-alleles 2 \
--max-missing 0.5 \
--recode \
--out subset_snps_no_indels_no_repeats_filtered

## Question: How many additional SNPs did this step remove? How many are left in your file?

### Answer here

20) Now that we have a cleaned up file, let's calculate how many reads we have for each SNP in each sample on average using the `depth` function of `vcftools`. Run the code below.

In [ ]:
%% bash
vcftools --vcf subset_snps_no_indels_no_repeats_filtered.recode.vcf --depth --out subset_snps_no_indels_no_repeats_filtered

This command created a new file called `subset_snps_no_indels_no_repeats_filtered.idepth` that contains the average number of reads at each SNP in the file (column is named `MEAN_DEPTH`)

## Question: Look at this file and report the mean depth of your two samples.

### Answer here:

## Question: how does this number compare to the sequencing coverage that you calculated in your original bam file near the beginning of class? 

### Answer here:

21) We are going to stop here today with our filtered vcf file, but before we go, do a mini writing assignment:  

## Question: Write a one-paragraph methods summary that you might include in a paper if you were writing this work from today up for a paper. Your paragraph should describe the important steps that were done and the software that was used.  You do not need to describe the steps that were just checking on things. But you should describe the SNP calling and filterning steps. 

### Answer here: 

## Check back over your work and make sure you have answered every question in the document before turning it in! I will be returning labs with unanswered questions for revisions this week. 

When you are done, go to file --> Download As --> PDF via LaTex. Save this PDF to your computer and then upload to the Blackboard assignment for this week's lab. 